In [1]:
import numpy as np
import pandas as pd

In [2]:
def get_dist_matrix(df_structures_idx, molecule):
    df_temp = df_structures_idx.loc[molecule]
    locs = df_temp[['x','y','z']].values
    num_atoms = len(locs)
    loc_tile = np.tile(locs.T, (num_atoms,1,1))
    dist_mat = np.linalg.norm(loc_tile - loc_tile.T, axis=1)
    return dist_mat

In [3]:
def c_cos(df_structures_idx, mol, thres=1.65):
    dist_mat = get_dist_matrix(df_structures_idx, mol)
    df_temp = df_structures_idx.loc[mol]
    num_atoms = df_temp.shape[0]

    c_idx = df_temp[df_temp['atom'] == 'C']['atom_index'].values

    c_cos2 = []
    c_cos3 = []

    for i in c_idx:
        dist_argsort = np.argsort(dist_mat[i])

        near_1_idx = dist_argsort[1]
        near_2_idx = dist_argsort[2]

        origin_loc = df_temp[df_temp['atom_index'] == i][['x', 'y', 'z']].values[0]
        near_1_loc = df_temp[df_temp['atom_index'] == near_1_idx][['x', 'y', 'z']].values[0]
        near_2_loc = df_temp[df_temp['atom_index'] == near_2_idx][['x', 'y', 'z']].values[0]

        vec_01 = near_1_loc - origin_loc
        vec_02 = near_2_loc - origin_loc
        
        cos_12 = np.dot(vec_01, vec_02) / dist_mat[i][near_1_idx] / dist_mat[i][near_2_idx] 
        
        c_cos2.append(cos_12)

        try:
            near_3_idx = dist_argsort[3]
            near_3_loc = df_temp[df_temp['atom_index'] == near_3_idx][['x', 'y', 'z']].values[0]
            vec_03 = near_3_loc - origin_loc
            
            if  dist_mat[i][near_3_idx] < thres:
                vec_012 = vec_01 / dist_mat[i][near_1_idx] + vec_02/ dist_mat[i][near_2_idx]
                cos_123 = np.dot(vec_012, vec_03) / np.linalg.norm(vec_012) /dist_mat[i][near_3_idx]
                c_cos3.append(cos_123)
            else:
                c_cos3.append(1)
        except:
             c_cos3.append(1)
            
    se_c_cos2 = pd.Series(c_cos2, name='cos2')
    se_c_cos3 = pd.Series(c_cos3, name='cos3')
    
    se_c_idx = pd.Series(c_idx, name='atom_index')
    df_bond = pd.concat([se_c_idx, se_c_cos2, se_c_cos3], axis=1)

    df_temp2 = pd.merge(df_temp[['atom', 'atom_index']], df_bond, on='atom_index', how='outer').fillna(1)
    df_temp2['molecule_name'] = mol
    return df_temp2

In [4]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [5]:
df = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
df_structures = pd.read_csv(FOLDER + 'structures.csv')

In [6]:
mols = df['molecule_name'].unique()

In [7]:
df_structures_idx = df_structures.set_index('molecule_name')
df_idx = df.set_index('molecule_name')

In [8]:
c_cos(df_structures_idx, mols[0], thres=1.65)

,atom,atom_index,cos2,cos3,molecule_name
0,C,0,-0.333342,-0.577373,dsgdb9nsd_000001
1,H,1,1.000000,1.000000,dsgdb9nsd_000001
2,H,2,1.000000,1.000000,dsgdb9nsd_000001
3,H,3,1.000000,1.000000,dsgdb9nsd_000001
4,H,4,1.000000,1.000000,dsgdb9nsd_000001


In [7]:
mat = get_dist_matrix(df_structures_idx, mols[0])
locs = df_structures_idx.loc[mols[0]][['x','y','z']].values

In [136]:
locs.shape

(5, 3)

In [102]:
sorted_mat_idx = np.argsort(mat)
sorted_mat_idx

array([[0, 3, 4, 2, 1],
       [1, 0, 2, 3, 4],
       [2, 0, 1, 4, 3],
       [3, 0, 1, 4, 2],
       [4, 0, 3, 2, 1]])

In [134]:
origin_idx = sorted_mat_idx[:, 0]
nearest_idx = sorted_mat_idx[:, 1]
second_idx = sorted_mat_idx[:, 2:]

In [135]:
second_idx

array([[4, 2, 1],
       [2, 3, 4],
       [1, 4, 3],
       [1, 4, 2],
       [3, 2, 1]])

In [110]:
origin_locs = locs[origin_idx]
nearest_locs = locs[nearest_idx]
second_locs = locs[second_idx]
base_vec = nearest_locs - origin_locs

In [127]:
base_tile = np.tile(base_vec.T, (3,1,1)).T
base_tile.shape

(5, 3, 3)

In [112]:
second_locs.shape

(5, 3, 3)

In [128]:
second_locs.shape

(5, 3, 3)

In [131]:
locs[second_idx][.shape

(5, 3, 3)

In [94]:
sorted_locs = locs[np.argsort(mat)]
origin = sorted_locs[:, 0]
nearest = sorted_locs[:, 1]
second = sorted_locs
base = nearest - origin
sec_vec = second - np.tile(origin, (5,1,1))
base_tile = np.tile(base, (5,1,1))
(base_tile * sec_vec).sum(axis=2) / np.linalg.norm(sec_vec, axis=2) / np.linalg.norm(base_tile, axis=2)

/home/ryoji/Documents/git/kaggle/ml/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


array([[        nan,  0.81649832,  0.81649835,  0.81650128,  0.81649981],
       [-0.33333494,  1.        ,         nan,         nan,         nan],
       [-0.33335189,  1.        ,  0.81648214,  0.81649909,  0.81649952],
       [ 1.        ,  1.        ,  0.81648214,  0.81649909,  0.81649682],
       [-0.33334224,  1.        ,  0.81650324,  0.81650128,  0.81649981]])

In [91]:
base_tile.shape

(5, 3, 3)

In [79]:
np.tile(base.T, (1,1,1)) .shape

(1, 3, 5)

In [145]:
sorted_locs[:, 2:][:,0]

array([[-5.23813635e-01,  1.43793264e+00,  9.06397294e-01],
       [ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
       [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
       [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
       [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01]])

In [141]:
locs

array([[-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
       [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
       [ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
       [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
       [-5.23813635e-01,  1.43793264e+00,  9.06397294e-01]])

In [35]:
np.tile(base, (5,1,1)).shape

(5, 5, 3)

In [28]:
(np.tile(base.T, (3,1,1)).T * sec_vec).sum(axis=1) / np.linalg.norm(sec_vec, axis=1) / np.linalg.norm(base, axis=1)

ValueError: operands could not be broadcast together with shapes (5,3) (5,) 

In [21]:

return (base * sec_vec).sum(axis=1) / np.linalg.norm(sec_vec, axis=1) / np.linalg.norm(base, axis=1)


array([[[-5.11115499e-01,  1.45063078e+00,  9.19095430e-01],
        [-7.40733150e-02,  3.77947004e-01, -1.08552758e+00],
        [-5.85057980e-03, -1.40323134e-02, -6.02487540e-03]],

       [[ 1.00958043e+00,  1.46160075e+00, -1.87384120e-03],
        [-5.34783751e-01,  1.45355793e+00, -8.70612398e-01],
        [-5.25789755e-01,  1.43595652e+00,  9.04421174e-01]],

       [[-1.00958043e+00, -1.01776216e+00, -1.00975472e+00],
        [-1.98756480e+00, -2.58185180e-02, -5.57353868e-01],
        [-5.41091644e-01,  1.44725004e+00, -8.76920290e-01]],

       [[ 5.42965485e-01,  5.34783751e-01,  5.42791189e-01],
        [-1.97134025e+00, -9.59397000e-03, -5.41129320e-01],
        [ 1.88837456e+00,  2.34039488e+00,  8.76920290e-01]],

       [[-1.70014345e-02,  1.97134025e+00, -3.52830081e-01],
        [-4.26201801e-01,  2.58185180e-02, -1.43765607e+00],
        [-9.04246878e-01, -9.12428612e-01, -9.04421174e-01]]])

In [174]:
get_angle(df_structures_idx, mols[0])

array([[ 1.        , -0.33334224, -0.33335189, -0.33333494],
       [ 1.        ,  0.81648268,  0.81649832,  0.81650188],
       [ 1.        ,  0.81648214,  0.81649835,  0.81650324],
       [ 1.        ,  0.81649582,  0.81649909,  0.81650128],
       [ 1.        ,  0.81649952,  0.81649682,  0.81649981]])

In [176]:
get_orientation(df_structures_idx, mols[0])

array([[ 1.        ,  1.        ,  0.50000392,  0.49999675],
       [ 1.        ,  1.        , -0.36236402, -0.36235604],
       [ 1.        ,  1.        , -0.36235995, -0.362357  ],
       [ 1.        ,  1.        , -0.36239269, -0.36233693],
       [ 1.        ,  1.        , -0.36239882, -0.36238805]])

In [173]:
def get_angle(df_structures_idx, mol):
    locs = df_structures_idx.loc[mol][['x','y','z']].values # ( ,3)
    mat = get_dist_matrix(df_structures_idx, mol)
    sorted_locs = locs[np.argsort(mat)]
    origin = sorted_locs[:, 0]
    nearest = sorted_locs[:, 1]
    second = sorted_locs[:, 2:]
    base = nearest - origin
    out_mat = np.zeros((0, len(mat)))
    for i in range(len(mat)-2):
        sec_vec = second[:,i,:] - origin
        out = (base * sec_vec).sum(axis=1) / np.linalg.norm(sec_vec, axis=1) / np.linalg.norm(base, axis=1)
        out_mat = np.vstack([out_mat, out])
    left = np.ones((len(mat), 1))
    return np.hstack([left, out_mat.T])

In [175]:
def get_orientation(df_structures_idx, mol):
    locs = df_structures_idx.loc[mol][['x','y','z']].values # ( ,3)
    mat = get_dist_matrix(df_structures_idx, mol)
    sorted_locs = locs[np.argsort(mat)]    
    
    origin = sorted_locs[:, 0]
    nearest = sorted_locs[:, 1]
    second = sorted_locs[:, 2]
    try:
        third = sorted_locs[:, 3:]
    except:
        return np.ones(len(sorted_locs))
    
    base = nearest - origin
    sec_vec = second - origin
    out_mat = np.zeros((0, len(mat)))
    left = np.ones((len(mat), 1))
    for i in range(len(mat)-3):
        thi_vec = third[:,i,:] - origin
        proj_1 = sec_vec - base * np.tile(np.linalg.norm(sec_vec, axis=1), (3,1)).T / np.tile(np.linalg.norm(base, axis=1), (3,1)).T
        proj_2 = thi_vec - base * np.tile(np.linalg.norm(thi_vec, axis=1), (3,1)).T / np.tile(np.linalg.norm(base, axis=1), (3,1)).T
        out = (proj_1*proj_2).sum(axis=1) / np.linalg.norm(proj_1, axis=1) / np.linalg.norm(proj_2, axis=1)
        out_mat = np.vstack([out_mat,out])
    return np.hstack([left, (np.hstack([left, out_mat.T]))])

In [12]:
get_orientation(df_structures_idx, mols[0])

array([ 0.50000392, -0.36236402, -0.36235995, -0.36239269, -0.36239882])

In [13]:
get_angle(df_structures_idx, mols[0])

array([-0.33334224,  0.81648268,  0.81648214,  0.81649582,  0.81649952])

In [14]:
mat

array([[0.        , 1.09195306, 1.09195162, 1.09194638, 1.09194754],
       [1.09195306, 0.        , 1.78311976, 1.7831475 , 1.78315669],
       [1.09195162, 1.78311976, 0.        , 1.78315766, 1.78314839],
       [1.09194638, 1.7831475 , 1.78315766, 0.        , 1.78314787],
       [1.09194754, 1.78315669, 1.78314839, 1.78314787, 0.        ]])

In [18]:
np.argsort(mat)

array([[0, 3, 4, 2, 1],
       [1, 0, 2, 3, 4],
       [2, 0, 1, 4, 3],
       [3, 0, 1, 4, 2],
       [4, 0, 3, 2, 1]])

In [19]:
locs[np.argsort(mat)][:, 0]

array([[-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
       [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
       [ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
       [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
       [-5.23813635e-01,  1.43793264e+00,  9.06397294e-01]])

In [20]:
locs[np.argsort(mat)]

array([[[-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
        [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
        [-5.23813635e-01,  1.43793264e+00,  9.06397294e-01],
        [ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
        [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03]],

       [[ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
        [-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
        [ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
        [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
        [-5.23813635e-01,  1.43793264e+00,  9.06397294e-01]],

       [[ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
        [-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
        [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
        [-5.23813635e-01,  1.43793264e+00,  9.06397294e-01],
        [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01]],

       [[-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
        [-1.269813

In [65]:
origin = test[:,0]
nearest = test[:,1]
second = test[:,2]
base = nearest - origin
sec_vec = second - origin

In [106]:
sec_vec

array([[-5.11115499e-01,  3.52128486e-01,  8.98396298e-01],
       [ 1.00958043e+00,  1.46978248e+00, -1.69954560e-03],
       [-1.00958043e+00, -1.46978248e+00,  1.69954560e-03],
       [ 5.42965485e-01, -1.45355793e+00,  8.78619836e-01],
       [-1.70014345e-02,  9.59397000e-03, -1.78304101e+00]])

In [124]:
np.tile(np.linalg.norm(sec_vec, axis=1), (3,1)).T

array([[1.09194754, 1.09194754, 1.09194754],
       [1.78311976, 1.78311976, 1.78311976],
       [1.78311976, 1.78311976, 1.78311976],
       [1.7831475 , 1.7831475 , 1.7831475 ],
       [1.78314787, 1.78314787, 1.78314787]])

In [70]:
(base * sec_vec).sum(axis=1) / np.linalg.norm(sec_vec, axis=1) / np.linalg.norm(base, axis=1)

array([-0.33334224,  0.81648268,  0.81648214,  0.81649582,  0.81649952])

In [72]:
get_angle(test) 

array([-0.33334224,  0.81648268,  0.81648214,  0.81649582,  0.81649952])

In [37]:
locs[np.argsort(mat)[:, :3]]

array([[[-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
        [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
        [-5.23813635e-01,  1.43793264e+00,  9.06397294e-01]],

       [[ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
        [-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
        [ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04]],

       [[ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
        [-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
        [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03]],

       [[-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
        [-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
        [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03]],

       [[-5.23813635e-01,  1.43793264e+00,  9.06397294e-01],
        [-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
        [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01]]])